# NLIL

In [1]:
from collections import defaultdict
import re

import pandas as pd

## 簡單分幾個column

In [2]:
raw = pd.read_json('data/news.json')

In [3]:
data = defaultdict(list)
columns = 'title time from content'.split()
pattern = ('(?P<title>.*|)'
           ' - 中時電子報\s*'
           '(?P<time>20[0-1][0-9]年[0-1][0-9]月[0-3][0-9]日 [0-2][0-9]:[0-5][0-9])'
           '(中國時報|中時及時) '
           '(?P<from>[^\n]*)'
           '\s*'
           '(?P<content>.*$)')
pattern = re.compile(pattern, flags=re.DOTALL)
for i, news in enumerate(raw.news, 1):
    match = pattern.search(news)
    if match is None: continue
    for col in columns:
        data[col].append(match.group(col))
data = pd.DataFrame(data)[columns]

## Split

- 根據有沒有出現法、律、條例、通則分成 `implicit` 和 `explicit`
- 應該要先針對 `explicit` 做點事 QAQ

In [8]:
related_words = {'法', '律', '條例', '通則'}
explicit = data.content.map(lambda s: any(word in s for word in related_words))
name = 'implicit', 'explicit'
for i, group in data.groupby(explicit):
    group.to_json('data/%s.jsonl'%name[i], lines=True, orient='records', force_ascii=False)

## 斷詞

- 先用 Jseg 斷一個， ckip 好麻煩

In [4]:
import json
from multiprocessing import Pool

from jseg.jieba import Jieba
segmenter = Jieba()

DEBUG:jseg.jieba:loading default dictionary


In [5]:
def pos_piece(piece):
    if piece['title']:
        title = segmenter.seg(piece['title'], pos=True).raw
    else:
        title = ''
    content = segmenter.seg(piece['content'], pos=True).raw
    piece['pos_title'] = title
    piece['pos_content'] = content

In [6]:
with open('data/explicit.jsonl', 'r') as f:
    data = [json.loads(piece) for piece in f]

In [7]:
for i, piece in enumerate(data, 1):
    pos_piece(piece)
    print('%5d/%5d'%(i, len(data)), end='\r')

In [10]:
with open('data/explicit_jseg.jsonl', 'w') as f:
    print(json.dumps(data, ensure_ascii=False), file=f)

---